Based on notebook 6, for further experiments only knn, xgboost and linear svc will be considered.  

#convert reason to numbers  
{'Mau Serviço Prestado': 0,  
'Condições de entrega': 1,   
'Atraso de entrega': 2,   
'Enganos': 3}  

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler #fixed import

import nltk
import re
from langdetect import detect
from scipy.sparse import hstack
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk import word_tokenize

In [2]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")
df.head()

,date,title,reason,description,zone,narrative_tfidf,narrative_embeddings,narrative_bert,narrative_tfidf_title,narrative_bert_title,...,tfidf_title,title_tfidf_nolemma,events,keywords,locs,embeddings_title,tfidf_keywords,embeddings_keywords,events_clean,events_tfidf
0,2021-10-11 12:26:32,CTT - Encomenda entregue danificada e com etiq...,0,Bom dia venho por este meio apresentar uma rec...,Lisboa,bom dia venho meio apresentar reclamação ctt e...,bom dia venho por este meio apresentar uma rec...,Bom dia venho por este meio apresentar uma rec...,ctt encomenda entregar danificar etiqueta envi...,ctt - encomenda entregue danificada e com etiq...,...,ctt encomenda entregar danificar etiqueta envi...,ctt encomenda entregue danificada etiquetas en...,"['venho', 'apresentar', 'Enviei', 'destinado',...",apresentar uma reclamação tinha caixa figura e...,"['Israel', 'Aboboda', 'Israel', 'de Braga', 'O...",ctt - encomenda entregue danificada e com etiq...,apresentar reclamação caixa figura envio israe...,apresentar uma reclamação tinha caixa figura e...,venho apresentar enviei destinado feito proteg...,quase destruir figura partida chegar Caixa che...
1,2021-08-31 23:20:15,CTT - Carta com certificado rasgado!,1,- Boa Noite!\r\n\r\n- Venho por este meio recl...,Coimbra,bom noite venho meio reclamar data ás durante ...,- boa noite!\r\n\r\n- venho por este meio recl...,- Boa Noite! Venho por este meio reclamar que ...,ctt carta certificar rasgar bom noite venho me...,ctt - carta com certificado rasgado! - Boa No...,...,ctt carta certificar rasgar,ctt carta certificado rasgado,"['Venho', 'reclamar', 'deparei -me', 'continha...",Lilia Pimentel certificado profissional passag...,['Boa Noite!'],ctt - carta com certificado rasgado !,lilia pimentel certificar profissional passage...,lilia pimentel certificado profissional passag...,venho reclamar deparei -me continha tem,NaN
2,2021-02-01 19:08:32,CTT - Encomenda registada que nunca chegou ao ...,2,"Boa tarde,\r\nNo dia 17 de Dezembro de 2020 en...",Aveiro,bom tarde dia dezembro enviar encomenda França...,"boa tarde,\r\nno dia 00 de dezembro de 0000 en...","Boa tarde, No dia 17 de Dezembro de 2020 envie...",ctt encomenda registar nunca chegar destine o ...,ctt - encomenda registada que nunca chegou ao ...,...,ctt encomenda registar nunca chegar destine o,ctt encomenda registada nunca chegou destino,"['enviei', 'registado', 'vejo', 'chegou', 'sai...",França em correio registado França em correio ...,"['França', 'Roissy', 'ROISSY']",ctt - encomenda registada que nunca chegou ao ...,frança correio registar frança correio encomen...,frança em correio registado frança em correio ...,enviei registado vejo chegou saiu chegou tem p...,resolr
3,2018-04-19 09:04:34,CTT - Correio Verde,2,"Bom dia,\n\nPretendia esclarecer uma situação ...",Lisboa,bom dia pretender esclarecer situação ocorrer ...,"bom dia,\n\npretendia esclarecer uma situação ...","Bom dia, Pretendia esclarecer uma situação que...",ctt correio verde bom dia pretender esclarecer...,"ctt - correio verde Bom dia, Pretendia esclar...",...,ctt correio verde,ctt correio verde,"['Pretendia esclarecer', 'ocorreu', 'desloquei...",Pretendia esclarecer uma situação Pretendia es...,['Lisboa'],ctt - correio verde,pretender esclarecer situação pretender esclar...,pretendia esclarecer uma situação pretendia es...,pretendia esclarecer ocorreu desloquei dado fe...,esclarecer saber semana enviar eventualmente a...
4,2019-07-04 10:20:41,"CTT - Aviso de recepção da carta registada, pr...",2,venho por este meio solicitar a vossa ajuda pa...,Porto,venho meio solicitar vosso ajudar saber onde e...,venho por este meio solicitar a vossa ajuda pa...,venho por este meio solicitar a vossa ajuda pa...,ctt aviso recepção carta registar preciso urge...,"ctt - aviso de recepção da carta registada, pr...",...,ctt aviso recepção carta registar preciso urge...,ctt aviso recepção carta registada preciso urg...,"['venho', 'solicitar', 'saber onde se encontra...",papais para o centro 

In [3]:
df['reason'].unique()

array([0, 1, 2, 3], dtype=int64)

In [4]:
# Create a Counter object
class_counts = Counter(df['reason'])

# Access the counts of each class
for class_label, count in class_counts.items():
    print(f"Class {class_label}: {count/len(df)} samples")

Class 0: 0.3551493535443602 samples
Class 1: 0.09893000445831476 samples
Class 2: 0.4782880071333036 samples
Class 3: 0.0676326348640214 samples


In [8]:
# before
# {'Mau Serviço Prestado': 0,
# 'Condições de entrega': 1,
# 'Atraso de entrega': 2,
# 'Enganos': 3}

# Map classes to new groups 
# keys correspond to the original class labels, and the values correspond to the new class labels
class_mapping = {
    0: 0,
    1: 0,
    2: 1,
    3: 0
}

# Step 3: Update the labels
df['reason_binary'] = df['reason'].map(class_mapping)

In [9]:
df['reason_binary'].unique()

array([0, 1], dtype=int64)

In [11]:
# Create a Counter object
class_counts = Counter(df['reason_binary'])

# Access the counts of each class
for class_label, count in class_counts.items():
    print(f"Class {class_label}: {count} samples")

Class 0: 11702 samples
Class 1: 10728 samples


# TF-IDF with description and title

In [12]:
# Separate features and labels
y = df['reason_binary']
X = df['narrative_tfidf_title']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

In [13]:
def train_classifier(dataset, X_train, X_val, y_train, y_val, clf):
    vectorizer = TfidfVectorizer(ngram_range=(1,3), max_df=0.95, min_df=2)
    X_train_tf = vectorizer.fit_transform(X_train)
    X_val_tf = vectorizer.transform(X_val)
    clf.fit(X_train_tf, y_train)
    y_val_pred = clf.predict(X_val_tf)
    
    confusion_mat = confusion_matrix(y_true=y_val, y_pred=y_val_pred, labels=[0,1])
    class_rep= classification_report(y_val, y_val_pred)
    
    print(class_rep)
    print(confusion_mat)
    
    # Create a DataFrame with the original texts, predicted labels, and true labels
    results_df = pd.DataFrame({'narrative_bert_title': dataset.loc[y_val.index, 'narrative_bert_title'],
                               'narrative_tfidf_title': dataset.loc[y_val.index, 'narrative_tfidf_title'],
                               'predicted label': y_val_pred,
                               'true label': y_val})

    misclassified_texts = results_df[results_df['predicted label'] != results_df['true label']]
    
    return misclassified_texts

# KNN

In [15]:
knn_misclassified_texts = train_classifier(df, X_train, X_val, y_train, y_val, KNeighborsClassifier())

              precision    recall  f1-score   support

           0       0.75      0.71      0.73      1755
           1       0.70      0.74      0.72      1609

    accuracy                           0.72      3364
   macro avg       0.72      0.72      0.72      3364
weighted avg       0.72      0.72      0.72      3364

[[1243  512]
 [ 419 1190]]


###  Misclassified texts

In [16]:
knn_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
738,ctt - falta de cumprimento na entrega do corre...,ctt falta cumprimento entregar correio nuno Ri...,0,1
16839,ctt - encomendas internacionais desaparecem um...,ctt encomenda internacional desaparecer um atr...,1,0
19777,ctt - correio Eu nao recebo o meu correio há ...,ctt correio nao receber correio semana carta s...,0,1
11659,ctt - reclamação centro de distribuição de alc...,ctt reclamação Centro distribuição Alcoitão sa...,1,0
21868,"ctt - encomendas nada ,correio nada ,o que se ...",ctt encomenda nada correio nada passar venho m...,1,0
...,...,...,...,...
20880,ctt - envio de encomenda em correio normal No...,ctt envio encomenda correio normal passar mês ...,0,1
7215,Boa Tarde Exmo(s) Venho por este meio solicita...,bom tarde exmo s venho meio solicitar esclarec...,1,0
21115,Venho por este meio reclamar que até à data nã...,venho meio reclamar data recebi três desempreg...,1,0
11903,ctt - encomenda recusada e ainda não devolvida...,ctt encomenda recusar ainda devolvir bom noite...,1,0


In [17]:
print('MISCLASSIFIED KNN TEXTS')
for i in range(100):
    print(knn_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(knn_misclassified_texts['true label'].iloc[i],
                                                                knn_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED KNN TEXTS
ctt - falta de cumprimento na entrega do correio  eu, Nuno ribeiro, venho por este meio, apresentar o meu descontentamento na entrega tardia da minha correspondência, por parte de alguns carteiros, que não tem condições psicológicas para estarem ao serviço e se desleixam na entrega do correio prejudicando as famílias que recebem apoios do estado para sua sobrevivência. 
TRUE LABEL IS 1 BUT IT WAS CLASSIFIED AS 0


ctt - encomendas internacionais desaparecem umas atrás das outras!  Onde estão as várias encomendas internacionais que fiz e nunca me chegaram a casa? Um mero exemplo entre muitos outros que podia dar, código de registo da encomenda: UC088173604YP esta encomenda chegou a Portugal a 24 de Abril, mas ao destinatário é que ela não chega, ou melhor: elas não chegam! Do que precisam para concluir as encomendas? Motivação? Uma palmadinha nas costas e uma frase amiga: Força vocês conseguem! - hum? Ou talvez precisem de formação para aprender a ser competentes

# XGBoost

In [18]:
xgb_misclassified_texts = train_classifier(df, X_train, X_val, y_train, y_val, XGBClassifier())

              precision    recall  f1-score   support

           0       0.79      0.75      0.77      1755
           1       0.74      0.78      0.76      1609

    accuracy                           0.77      3364
   macro avg       0.77      0.77      0.77      3364
weighted avg       0.77      0.77      0.77      3364

[[1318  437]
 [ 347 1262]]


###  Misclassified texts

In [19]:
xgb_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
738,ctt - falta de cumprimento na entrega do corre...,ctt falta cumprimento entregar correio nuno Ri...,0,1
16839,ctt - encomendas internacionais desaparecem um...,ctt encomenda internacional desaparecer um atr...,1,0
19777,ctt - correio Eu nao recebo o meu correio há ...,ctt correio nao receber correio semana carta s...,0,1
11659,ctt - reclamação centro de distribuição de alc...,ctt reclamação Centro distribuição Alcoitão sa...,1,0
21868,"ctt - encomendas nada ,correio nada ,o que se ...",ctt encomenda nada correio nada passar venho m...,1,0
...,...,...,...,...
4358,ctt - cartas registadas No dia 3.01.2020 fui ...,ctt carta registar dia ctt Fernão ferro entreg...,1,0
8500,ctt - encomenda não entregue Em 21/06 enviei ...,ctt encomenda entregar enviar encomenda nº irm...,0,1
5184,ctt - endereço incorrecto ou insuficiente Fiz...,ctt endereço incorrecto insuficiente fiz encom...,1,0
21115,Venho por este meio reclamar que até à data nã...,venho meio reclamar data recebi três desempreg...,1,0


In [20]:
print('MISCLASSIFIED XGB TEXTS')
for i in range(100):
    print(xgb_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(xgb_misclassified_texts['true label'].iloc[i],
                                                                xgb_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED XGB TEXTS
ctt - falta de cumprimento na entrega do correio  eu, Nuno ribeiro, venho por este meio, apresentar o meu descontentamento na entrega tardia da minha correspondência, por parte de alguns carteiros, que não tem condições psicológicas para estarem ao serviço e se desleixam na entrega do correio prejudicando as famílias que recebem apoios do estado para sua sobrevivência. 
TRUE LABEL IS 1 BUT IT WAS CLASSIFIED AS 0


ctt - encomendas internacionais desaparecem umas atrás das outras!  Onde estão as várias encomendas internacionais que fiz e nunca me chegaram a casa? Um mero exemplo entre muitos outros que podia dar, código de registo da encomenda: UC088173604YP esta encomenda chegou a Portugal a 24 de Abril, mas ao destinatário é que ela não chega, ou melhor: elas não chegam! Do que precisam para concluir as encomendas? Motivação? Uma palmadinha nas costas e uma frase amiga: Força vocês conseguem! - hum? Ou talvez precisem de formação para aprender a ser competentes

# SVC

In [21]:
svc_misclassified_texts = train_classifier(df, X_train, X_val, y_train, y_val, LinearSVC())

              precision    recall  f1-score   support

           0       0.80      0.78      0.79      1755
           1       0.77      0.79      0.78      1609

    accuracy                           0.78      3364
   macro avg       0.78      0.78      0.78      3364
weighted avg       0.78      0.78      0.78      3364

[[1373  382]
 [ 345 1264]]


###  Misclassified texts

In [22]:
svc_misclassified_texts

,narrative_bert_title,narrative_tfidf_title,predicted label,true label
738,ctt - falta de cumprimento na entrega do corre...,ctt falta cumprimento entregar correio nuno Ri...,0,1
16839,ctt - encomendas internacionais desaparecem um...,ctt encomenda internacional desaparecer um atr...,1,0
19777,ctt - correio Eu nao recebo o meu correio há ...,ctt correio nao receber correio semana carta s...,0,1
11659,ctt - reclamação centro de distribuição de alc...,ctt reclamação Centro distribuição Alcoitão sa...,1,0
21868,"ctt - encomendas nada ,correio nada ,o que se ...",ctt encomenda nada correio nada passar venho m...,1,0
...,...,...,...,...
7215,Boa Tarde Exmo(s) Venho por este meio solicita...,bom tarde exmo s venho meio solicitar esclarec...,1,0
17084,ctt - falta de dados Venho por este meio recl...,ctt falta dar venho meio reclamar relação enco...,1,0
21115,Venho por este meio reclamar que até à data nã...,venho meio reclamar data recebi três desempreg...,1,0
11903,ctt - encomenda recusada e ainda não devolvida...,ctt encomenda recusar ainda devolvir bom noite...,1,0


In [23]:
print('MISCLASSIFIED SVC TEXTS')
for i in range(100):
    print(svc_misclassified_texts['narrative_bert_title'].iloc[i])
    print('TRUE LABEL IS {} BUT IT WAS CLASSIFIED AS {}'.format(svc_misclassified_texts['true label'].iloc[i],
                                                                svc_misclassified_texts['predicted label'].iloc[i]))
    print('\n')

MISCLASSIFIED SVC TEXTS
ctt - falta de cumprimento na entrega do correio  eu, Nuno ribeiro, venho por este meio, apresentar o meu descontentamento na entrega tardia da minha correspondência, por parte de alguns carteiros, que não tem condições psicológicas para estarem ao serviço e se desleixam na entrega do correio prejudicando as famílias que recebem apoios do estado para sua sobrevivência. 
TRUE LABEL IS 1 BUT IT WAS CLASSIFIED AS 0


ctt - encomendas internacionais desaparecem umas atrás das outras!  Onde estão as várias encomendas internacionais que fiz e nunca me chegaram a casa? Um mero exemplo entre muitos outros que podia dar, código de registo da encomenda: UC088173604YP esta encomenda chegou a Portugal a 24 de Abril, mas ao destinatário é que ela não chega, ou melhor: elas não chegam! Do que precisam para concluir as encomendas? Motivação? Uma palmadinha nas costas e uma frase amiga: Força vocês conseguem! - hum? Ou talvez precisem de formação para aprender a ser competentes

# Embeddings with description and title

In [ ]:
w2v= KeyedVectors.load_word2vec_format('D:\\MS DATA SCIENCE\\NLP TESE\\embeddings\\skip_s600_word2vec.txt')
glove= KeyedVectors.load_word2vec_format('D:\\MS DATA SCIENCE\\NLP TESE\\embeddings\\glove_s600.txt')

In [ ]:
def getVectors(dataset, model_name, number_of_features): 
    # Creating empty dataframe to hold sentences
    W2Vec_Data=pd.DataFrame()
    for review in dataset:
        wordCount=0
        # initiating a sentence with all zeros
        word_Embedding=np.zeros(number_of_features)
        
        # Loop over each word in the review and, if it is in the model's vocaublary, add its feature vector to the total
        for word in word_tokenize(review.lower()): 
            if word in model_name.key_to_index.keys(): # model_w2v.wv.vocab is a list that contains the names of the words in the model's vocabulary
                word_Embedding=word_Embedding+model_name[word] 
                wordCount=wordCount+1
                
        # Appending the sentence to the dataframe
        W2Vec_Data=W2Vec_Data.append(pd.DataFrame([word_Embedding/wordCount]))
    return W2Vec_Data

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Calling the function to convert all the text data to Word2Vec Vectors
concatenated_w2v=getVectors(df['narrative_embeddings_title'],w2v,600)

# Checking the new representation for sentences
print(concatenated_w2v.shape)
print(concatenated_w2v.head())

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Calling the function to convert all the text data to Word2Vec Vectors
concatenated_glove=getVectors(df['narrative_embeddings_title'],glove,600)

# Checking the new representation for sentences
print(concatenated_glove.shape)
print(concatenated_glove.head())